In [54]:
import os
import psycopg2

from openai import AzureOpenAI
from dotenv import load_dotenv

from pgvector.psycopg2 import register_vector

load_dotenv()

client = AzureOpenAI(
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

conn = psycopg2.connect(os.getenv("POSTGRES_CONFIG"))
register_vector(conn)
cursor = conn.cursor()


def get_embeddings_ada(input):
    response = client.embeddings.create(
        input = input,
        model= "text-embedding-ada-002"
    )

    return response.data[0].embedding

def get_resource_id(cursor, resource_name):
    query = """SELECT * FROM resource WHERE name = %s;"""
    cursor.execute(query, (resource_name,))
    result = cursor.fetchone()
    
    return result[0]

def get_retrieved_knowledge(cursor, query, resource_ids, top_k):
    query = """
SELECT 
    c.id,
    c.fact AS content,
    json_build_object('number', c.number, 'summary', c.summary, 'filetype', m.filetype, 'name', r.name)::jsonb AS metadata,
    1 - (c.embeddings <-> %(query_embedding)s::vector) AS similarity
FROM fact c JOIN resource r ON c.resource_id = r.id JOIN metadata m ON r.id = m.resource_id
WHERE c.resource_id = ANY (%(resource_ids)s::UUID[])
ORDER BY c.embeddings <-> %(query_embedding)s::vector
LIMIT %(match_count)s;
"""
    cursor.execute(query, {
    "query_embedding": get_embeddings_ada(query),
    "resource_ids": resource_ids,
    "match_count": top_k,
})
    result = cursor.fetchall()

    return result

def delete_facts_resource(conn, cursor, resource_id):
    delete_query = """
    DELETE FROM fact WHERE resource_id = %s;
    """
    cursor.execute(delete_query, (resource_id,))
    conn.commit()

def insert_fact_resource(conn, cursor, data):
    insert_query = """
    INSERT INTO fact (context, fact, resource_id, embeddings, summary, number)
    VALUES (%(context)s, %(fact)s, %(resource_id)s, %(embeddings)s, %(summary)s, %(number)s);
    """
    cursor.execute(insert_query, data)
    conn.commit()

In [52]:
query = """
SELECT 
    c.id,
    c.fact AS content,
    json_build_object('number', c.number, 'summary', c.summary, 'filetype', m.filetype, 'name', r.name)::jsonb AS metadata,
    1 - (c.embeddings <-> %(query_embedding)s::vector) AS similarity
FROM fact c JOIN resource r ON c.resource_id = r.id JOIN metadata m ON r.id = m.resource_id
WHERE c.resource_id = ANY (%(resource_ids)s::UUID[])
ORDER BY c.embeddings <-> %(query_embedding)s::vector
LIMIT %(match_count)s;
"""
cursor.execute(query, {
    "query_embedding": get_embeddings_ada("Siapakah Ketua BPK saat ini"),
    "resource_ids": [get_resource_id(cursor, "Test BPK - NonLLM")],
    "match_count": 5,
})

In [53]:
result = cursor.fetchall()
for res in result:
    print(res)

('c94259a9-8a83-49e8-8aa6-2a6dcc1e7eed', 'A. Pimpinan Badan Pemeriksa Keuangan Republik Indonesia (BPK RI) saat ini adalah sebagai berikut:\nKetua BPK: Dr. Isma Yatun, CSFA., CFrA. \nWakil Ketua BPK: Dr. Agus Joko Pramono, M.Acc., Ak., CA., CSFA., CPA., CFrA., QGIA., CGCAE.\nAnggota I BPK: Nyoman Adhi Suryadnyana, SE., ME., M.Ak., CSFA., CertDA., CGCAE.\nAnggota II BPK: Ir. Daniel Lumban Tobing, CSFA., CFrA.\nAnggota III BPK: Prof. Dr. Achsanul Qosasi, CSFA., CFrA., CGCAE.\nAnggota IV BPK: Haerul Saleh, SH., CRA., CRP., CIABV.\nAnggota V BPK: Ir. H. Ahmadi Noor Supit, MM.\nAnggota VI BPK: Dr. Pius Lustrilanang S.IP., M.Si., CSFA., CFrA.\nAnggota VII BPK: Dr. Hendra Susanto, ST., M.Eng., MH., CFrA., CSFA., CIAE., CGCAE., CertDA.\nB. Tugas dan Wewenang Pimpinan Badan Pemeriksa Keuangan Republik Indonesia (BPK RI) saat ini adalah sebagai berikut:\n a. Tugas dan Wewenang Ketua BPK:\n  i. Melaksanakan pemeriksaan pengelolaan dan tanggung jawab keuangan negara secara umum bersama dengan Waki